E1 Mastery

Brian Bannon

The purpose of this analysis is to evaluate and compare multiple solar panel system options for a residential client in Denver, Colorado. Real-world solar design requires balancing several competing factors, including upfront installation cost, available roof area, long-term energy production, system degradation, and financial return over time. Different clients may prioritize minimizing cost, maximizing energy production, or achieving the greatest long-term value from their investment.<br>
In this report, a set of Python functions is developed to model solar system performance and financial outcomes over a 25-year system lifetime. These functions calculate system specifications, yearly energy production with degradation, cumulative financial value with increasing electricity rates, and identify the optimal panel option based on different client priorities. Three panel options—budget, standard, and premium—are analyzed under identical site conditions to allow for a fair comparison.<br>
The results of this analysis provide quantitative insight into energy output, payback period, lifetime cost-effectiveness, and overall value, supporting an informed engineering recommendation for the client.


In [1]:
def calculate_system_specs(available_area: float, panel_area: float, panel_wattage: float, peak_sun_hours: float, cost_per_panel: float, installation_cost_per_panel: float, fixed_cost: float) -> list[float]:
    """
    Function to calculate 4 system specs
    :param available_area: the available area on the roof
    :param panel_area: the size of a panel
    :param panel_wattage: the wattage of a panel
    :param peak_sun_hours: the number of peak sun hours
    :param cost_per_panel: the cost of one panel
    :param installation_cost_per_panel: the cost to install one panel
    :param fixed_cost: fixed installation costs
    :return: the number of panels, the total cost, the daily production and the annual production
    """
    num_panels = available_area // panel_area
    total_cost = num_panels * (cost_per_panel + installation_cost_per_panel) + fixed_cost
    daily_kwh = num_panels * panel_wattage * peak_sun_hours / 1000
    annual_kwh = daily_kwh * 365
    return [num_panels, total_cost, daily_kwh, annual_kwh]

In [2]:
def calculate_yearly_production(initial_annual_kwh: float, degradation_rate: float, years: int) -> list[float]:
    """
    Function to calculate yearly production during the entire system lifetime
    :param initial_annual_kwh: The yearly production during the first year
    :param degradation_rate: How much the panels degrade per year
    :param years: the number of years to calculate
    :return: a list of production values for each year
    """
    output = []
    production = initial_annual_kwh
    degradation = 1
    for i in range(years):
        output.append(production * degradation)
        degradation -= degradation_rate
    return output

In [3]:


def calculate_cumulative_value(yearly_production: list[float], electricity_rate: float, rate_increase: float, total_cost: float) -> list[float]:
    """
    Function to calculate the cumulative value of the solar panels
    :param yearly_production: list of annual production for each year
    :param electricity_rate: Current cost of electricity
    :param rate_increase: Annual rate increase
    :param total_cost: Total installation cost
    :return: A list of the cumulative net value for each year
    """
    cumulative_savings = 0
    output = []
    for i in yearly_production:
        cumulative_savings += i * electricity_rate
        output.append(cumulative_savings - total_cost)
        electricity_rate *= 1 + rate_increase
    return output

In [4]:
def find_best_option(options: list[str], specs_list: list[list[float]], values_list: list[list[float]], priority: str) -> str:
    """
    Function to find the best option given a priority
    :param options: the names of each option
    :param specs_list: the specs of each option
    :param values_list: the values overtime of each option
    :param priority: the priority to determine which option is the best
    :return: the name of the best option
    """
    if priority == 'cost':
        output = options[0]
        lowest_cost = specs_list[0][1]
        for option, specs in zip(options, specs_list):
            if specs[1] < lowest_cost:
                lowest_cost = specs[1]
                output = option
        return output
    elif priority == 'production':
        output = options[0]
        highest_annual = specs_list[0][3]
        for option, specs in zip(options, specs_list):
            if specs[3] < highest_annual:
                highest_annual = specs[3]
                output = option
        return output
    else:
        output = options[0]
        most_value = values_list[0][-1]
        for option, values in zip(options, values_list):
            if values[-1] > most_value:
                most_value = values[-1]
                output = option
        return output

In [5]:
# Alternate common parameters
area = 800
hours = 5.5
fixed = 3500
panel_installation = 140
rate = 0.2
r_increase = 0.035
lifetime = 25

In [6]:
# Common parameters
area = 800
hours = 5.5
fixed = 3500
panel_installation = 140
rate = 0.14
r_increase = 0.035
lifetime = 25

In [7]:
def print_info(name):
    specs = calculate_system_specs(area, p_area, p_wattage, hours, panel_cost, panel_installation, fixed)
    num_panels, total_cost, daily_kwh, annual_kwh = specs
    yearly_production = calculate_yearly_production(annual_kwh, d_rate, lifetime)
    cumulative_value = calculate_cumulative_value(yearly_production, rate, r_increase, total_cost)
    payback_year = None
    for i, j in enumerate(cumulative_value):
        if j >= 0:
            payback_year = i + 1
            break
    total_production = sum(yearly_production)
    cost_per_lifetime_kwh = total_cost / total_production
    left_spacing = 30
    right_spacing = 9
    print(f'{name}\n'
          f'{'Number of panels':<{left_spacing}}{num_panels:>{right_spacing}.0f}\n'
          f'{'Total cost':<{left_spacing}}{f'${total_cost:,.0f}':>{right_spacing}}\n'
          f'{'Daily production (kwh)':<{left_spacing}}{f'{daily_kwh:.0f}':>{right_spacing}}\n'
          f'{'Year 1 production (kwh)':<{left_spacing}}{f'{yearly_production[0]:,.0f}':>{right_spacing}}\n'
          f'{'Final year production (kwh)':<{left_spacing}}{f'{yearly_production[-1]:,.0f}':>{right_spacing}}\n'
          f'{'Total energy production (kwh)':<{left_spacing}}{f'{total_production:,.0f}':>{right_spacing}}\n'
          f'{'Production decline':<{left_spacing}}{f'{(yearly_production[0] - yearly_production[-1])/yearly_production[0]*100:,.0f}%':>{right_spacing}}\n'
          f'{'Payback year':<{left_spacing}}{f'{payback_year}':>{right_spacing}}\n'
          f'{'Cumulative value at year 10':<{left_spacing}}{f'${cumulative_value[9]:,.0f}':>{right_spacing}}\n'
          f'{'Final net value':<{left_spacing}}{f'${cumulative_value[-1]:,.0f}':>{right_spacing}}\n'
          f'{'Lifetime cost per kWh':<{left_spacing}}{f'${cost_per_lifetime_kwh:,.2f}':>{right_spacing}}\n')
    return specs, cumulative_value

In [8]:
name1 = 'Budget Panels'
p_area = 18.5
p_wattage = 310
panel_cost = 180
d_rate = 0.008
specs1, values1 = print_info(name1)

Budget Panels
Number of panels                     43
Total cost                      $17,260
Daily production (kwh)               73
Year 1 production (kwh)          26,760
Final year production (kwh)      21,622
Total energy production (kwh)   604,775
Production decline                  19%
Payback year                          5
Cumulative value at year 10     $25,009
Final net value                $112,590
Lifetime cost per kWh             $0.03



In [9]:
name2 = 'Standard Panels'
p_area = 17.5
p_wattage = 370
panel_cost = 285
d_rate = 0.005
specs2, values2 = print_info(name2)

Standard Panels
Number of panels                     45
Total cost                      $22,625
Daily production (kwh)               92
Year 1 production (kwh)          33,425
Final year production (kwh)      29,414
Total energy production (kwh)   785,485
Production decline                  12%
Payback year                          5
Cumulative value at year 10     $30,959
Final net value                $147,094
Lifetime cost per kWh             $0.03



In [10]:
name3 = 'Premium Panels'
p_area = 17
p_wattage = 420
panel_cost = 395
d_rate = 0.003
specs3, values3 = print_info(name3)

Premium Panels
Number of panels                     47
Total cost                      $28,645
Daily production (kwh)              109
Year 1 production (kwh)          39,628
Final year production (kwh)      36,775
Total energy production (kwh)   955,036
Production decline                   7%
Payback year                          5
Cumulative value at year 10     $35,506
Final net value                $178,521
Lifetime cost per kWh             $0.03



In [11]:
best_cost = find_best_option([name1, name2, name3], [specs1, specs2, specs3], [values1, values2, values3], 'cost')
best_production = find_best_option([name1, name2, name3], [specs1, specs2, specs3], [values1, values2, values3], 'production')
best_value = find_best_option([name1, name2, name3], [specs1, specs2, specs3], [values1, values2, values3], 'value')
print('Best cost:', best_cost)
print('Best production:', best_production)
print('Best value:', best_value)

Best cost: Budget Panels
Best production: Budget Panels
Best value: Premium Panels


QUESTION 1<br>
Option 3 shows the smallest production decline (7%)

QUESTION 2<br>
Option 3 has the best value after 10 years. I would still recommend option 3 since it has the best value and production.

QUESTION 3<br>
All 3 options deliver electricity at the same rate of $0.03 which is more than 4 smaller than the current utility rate.

QUESTION 4<br>
Increasing the rate to $0.20 makes the payback periods one year shorter.

QUESTION 5<br>
I would recommend the third option. All 3 options pay for themselves after 5 years but the third option has the highest value by far and. Even if you do end up moving in 10 years, this option is still the most valuable.

This analysis compared three solar panel options using consistent site conditions and a 25-year system lifetime. While the budget panels offered the lowest upfront cost, the premium panels consistently outperformed the other options in terms of energy production, lowest degradation rate, and final cumulative value. All three systems achieved a payback period of approximately five years, indicating that solar is a financially viable investment for this client regardless of panel choice.<br>
When considering long-term performance, the premium panel option produced the most total energy, experienced the smallest production decline over time, and delivered the highest net financial value both at year 10 and at the end of the system lifetime. Additionally, all options generated electricity at a significantly lower lifetime cost per kilowatt-hour than the current utility rate, reinforcing the economic advantage of solar energy.<br>
Based on the results and the client’s uncertainty about moving within 10 years, the premium panel option is recommended. It provides the strongest combination of early value, long-term return, and maximum energy production, making it the most robust and future-proof choice among the options evaluated.
